In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd
import tensorflow as tf

from matplotlib import pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, AveragePooling2D, Lambda, Input, Dense, Flatten
from tensorflow.keras.layers import concatenate, BatchNormalization
from tensorflow.keras.layers import ReLU, LeakyReLU, PReLU, Activation
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.regularizers import l2

In [ ]:
optimizer = SGD(learning_rate=1e-2, momentum=9e-1)
weight_decay = 5e-4


# vgg with leaky relu
def vgg_net16_lr(input_shape=(224, 224, 3), classes=None, alpha=0.3):
    # input layer
    input_layer = Input(shape=input_shape, name='input_')

    # first conv block
    x = Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), kernel_regularizer=l2(weight_decay), padding='same')(
        input_layer)
    x = LeakyReLU(alpha=alpha)
    x = Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), kernel_regularizer=l2(weight_decay), padding='same')(x)
    x = LeakyReLU(alpha=alpha)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(x)

    # second conv block
    x = Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1), kernel_regularizer=l2(weight_decay), padding='same')(x)
    x = LeakyReLU(alpha=alpha)
    x = Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1), kernel_regularizer=l2(weight_decay), padding='same')(x)
    x = LeakyReLU(alpha=alpha)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(x)

    # third conv block
    x = Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), kernel_regularizer=l2(weight_decay), padding='same')(x)
    x = LeakyReLU(alpha=alpha)
    x = Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), kernel_regularizer=l2(weight_decay), padding='same')(x)
    x = LeakyReLU(alpha=alpha)
    x = Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), kernel_regularizer=l2(weight_decay), padding='same')(x)
    x = LeakyReLU(alpha=alpha)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(x)

    # fourth conv block
    x = Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1), kernel_regularizer=l2(weight_decay), padding='same')(x)
    x = LeakyReLU(alpha=alpha)
    x = Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1), kernel_regularizer=l2(weight_decay), padding='same')(x)
    x = LeakyReLU(alpha=alpha)
    x = Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1), kernel_regularizer=l2(weight_decay), padding='same')(x)
    x = LeakyReLU(alpha=alpha)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(x)

    # fifth conv block
    x = Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1), kernel_regularizer=l2(weight_decay), padding='same')(x)
    x = LeakyReLU(alpha=alpha)
    x = Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1), kernel_regularizer=l2(weight_decay), padding='same')(x)
    x = LeakyReLU(alpha=alpha)
    x = Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1), kernel_regularizer=l2(weight_decay), padding='same')(x)
    x = LeakyReLU(alpha=alpha)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(x)

    # classifier
    x = Flatten()(x)
    x = Dense(units=512)(x)
    x = LeakyReLU(alpha=alpha)
    x = Dropout(rate=0.5)(x)
    x = Dense(units=512)(x)
    x = LeakyReLU(alpha=alpha)
    x = Dropout(rate=0.5)(x)
    x = Dense(units=classes, activation='softmax')(x)

    model = Model(input_layer, x)
    model.compile(optimizer=optimizer, loss=categorical_crossentropy, metrics=['accuracy'])
    # model.summary()

    return model

In [ ]:
# inception with leaky relu
def inception(x, filters, projection, classes=None, aux=False, name=None, aux_name=None, alpha=0.3):
    f_1x1, f_3x3, f_3x3_reduce, f_5x5, f_5x5_reduce = filters
    x1 = Conv2D(filters=f_1x1, kernel_size=(1, 1), strides=(1, 1), padding='same')(x)
    x1 = LeakyReLU(alpha=alpha)(x1)
    x3_reducer = Conv2D(filters=f_3x3_reduce, kernel_size=(1, 1), strides=(1, 1), padding='same')(x)
    x3_reducer = LeakyReLU(alpha=alpha)(x3_reducer)
    x5_reducer = Conv2D(filters=f_5x5_reduce, kernel_size=(1, 1), strides=(1, 1), padding='same')(x)
    x5_reducer = LeakyReLU(alpha=alpha)(x5_reducer)
    pool = MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(x)

    x3 = Conv2D(filters=f_3x3, kernel_size=(3, 3), strides=(1, 1), padding='same')(x3_reducer)
    x3 = LeakyReLU(alpha=alpha)(x3)
    x5 = Conv2D(filters=f_5x5, kernel_size=(5, 5), strides=(1, 1), padding='same')(x5_reducer)
    x5 = LeakyReLU(alpha=alpha)(x5)
    proj = Conv2D(filters=projection, kernel_size=(1, 1), strides=(1, 1), padding='same')(pool)
    proj = LeakyReLU(alpha=alpha)(proj)

    x = concatenate([x1, x3, x5, proj], axis=3, name=name)

    return x


def model_builder(shape, classes, alpha=0.3):
    input_layer = Input(shape=shape)
    x = Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2), padding='same')(input_layer)
    x = LeakyReLU(alpha=alpha)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters=64, kernel_size=(1, 1), strides=(1, 1), padding='same')(x)
    x = LeakyReLU(alpha=alpha)
    x = Conv2D(filters=192, kernel_size=(3, 3), strides=(1, 1), padding='same')(x)
    x = LeakyReLU(alpha=alpha)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = inception(x, [64, 128, 96, 32, 16], projection=32, alpha=alpha, name='inception_3a')
    x = inception(x, [128, 192, 128, 96, 32], projection=64, alpha=alpha, name='inception_3b')
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = inception(x, [192, 208, 96, 48, 16], projection=64, alpha=alpha, name='inception_4a')

    aux_1 = AveragePooling2D(pool_size=(5, 5), strides=(3, 3), padding='valid')(x)
    aux_1 = Conv2D(filters=128, kernel_size=(1, 1), strides=(1, 1), padding='valid')(aux_1)
    aux_1 = LeakyReLU(alpha=alpha)
    aux_1 = Dense(units=1024)(aux_1)
    aux_1 = LeakyReLU(alpha=alpha)
    aux_1 = Dropout(rate=0.7)(aux_1)
    aux_1 = Flatten()(aux_1)
    aux_out1 = Dense(units=classes, activation='softmax', name='aux_out1')(aux_1)

    x = inception(x, [160, 224, 112, 64, 24], projection=64, alpha=alpha, name='inception_4b')
    x = inception(x, [128, 256, 128, 64, 24], projection=64, alpha=alpha, name='inception_4c')
    x = inception(x, [112, 288, 144, 64, 32], projection=64, alpha=alpha, name='inception_4d')
    x = inception(x, [256, 320, 160, 128, 32], projection=128, alpha=alpha, name='inception_4e')

    aux_2 = AveragePooling2D(pool_size=(5, 5), strides=(3, 3), padding='valid')(x)
    aux_2 = Conv2D(filters=128, kernel_size=(1, 1), strides=(1, 1), padding='valid')(aux_2)
    aux_2 = LeakyReLU(alpha=alpha)
    aux_2 = Dense(units=1024)(aux_2)
    aux_2 = LeakyReLU(alpha=alpha)
    aux_2 = Dropout(rate=0.7)(aux_2)
    aux_2 = Flatten()(aux_2)
    aux_out2 = Dense(units=classes, activation='softmax', name='aux_out2')(aux_2)

    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = inception(x, [256, 320, 160, 128, 32], projection=128, alpha=alpha, name='inception_5a')
    x = inception(x, [384, 384, 192, 128, 48], projection=128, alpha=alpha, name='inception_5b')
    x = AveragePooling2D(pool_size=(7, 7), strides=(1, 1))(x)
    x = Dropout(rate=0.4)(x)
    x = Flatten()(x)
    output_layer = Dense(units=classes, activation='softmax', name='main_out')(x)

    model = Model(input_layer, [output_layer, aux_out1, aux_out2])
    model.compile(optimizer=Adam(), loss=categorical_crossentropy,
                  loss_weights={'main_out': 1, 'aux_out1': 0.3, 'aux_out2': 0.3},
                  metrics=['accuracy'])
    # model.summary()

    return model

In [ ]:
vgg_a1 = vgg_net16_lr(input_shape=(96, 96, 3), classes=121, alpha=0.01)
vgg_a2 = vgg_net16_lr(input_shape=(96, 96, 3), classes=121, alpha=0.3)
vgg_a3 = vgg_net16_lr(input_shape=(96, 96, 3), classes=121, alpha=1.0)
vgg_a4 = vgg_net16_lr(input_shape=(96, 96, 3), classes=121, alpha=3.3)
vgg_a5 = vgg_net16_lr(input_shape=(96, 96, 3), classes=121, alpha=10.0)

In [ ]:
inception_a1 = model_builder(shape=(96, 96, 3), classes=121, alpha=0.01)
inception_a2 = model_builder(shape=(96, 96, 3), classes=121, alpha=0.3)
inception_a3 = model_builder(shape=(96, 96, 3), classes=121, alpha=1.0)
inception_a4 = model_builder(shape=(96, 96, 3), classes=121, alpha=3.3)
inception_a5 = model_builder(shape=(96, 96, 3), classes=121, alpha=10.0)

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.1,
                              patience=10)

In [ ]:
generator = ImageDataGenerator(rescale=1 / 255.,
                               vertical_flip=True,
                               horizontal_flip=True,
                               rotation_range=20,
                               zoom_range=0.3,
                               validation_split=0.2)

train_batch = generator.flow_from_directory(directory='../input/ndsb-training-data/train/',
                                            target_size=(96, 96),
                                            subset='training')

valid_batch = generator.flow_from_directory(directory='../input/ndsb-training-data/train/',
                                            target_size=(96, 96),
                                            subset='validation')

## ***VGGNet training***

In [ ]:
history_a1 = vgg_a1.fit(train_batch,
                        epochs=50,
                        validation_data=valid_batch,
                        steps_per_epoch=256,
                        callbacks=[reduce_lr])

pd.DataFrame(history_a1.history).to_pickle('history_a1.pkl')

In [ ]:
history_a2 = vgg_a2.fit(train_batch,
                        epochs=50,
                        validation_data=valid_batch,
                        steps_per_epoch=256,
                        callbacks=[reduce_lr])

pd.DataFrame(history_a2.history).to_pickle('history_a2.pkl')

In [ ]:
history_a3 = vgg_a3.fit(train_batch,
                        epochs=50,
                        validation_data=valid_batch,
                        steps_per_epoch=256,
                        callbacks=[reduce_lr])

pd.DataFrame(history_a3.history).to_pickle('history_a3.pkl')

In [ ]:
history_a4 = vgg_a4.fit(train_batch,
                        epochs=50,
                        validation_data=valid_batch,
                        steps_per_epoch=256,
                        callbacks=[reduce_lr])

pd.DataFrame(history_a4.history).to_pickle('history_a4.pkl')

In [ ]:
history_a5 = vgg_a5.fit(train_batch,
                        epochs=50,
                        validation_data=valid_batch,
                        steps_per_epoch=256,
                        callbacks=[reduce_lr])

pd.DataFrame(history_a5.history).to_pickle('history_a5.pkl')

## ***Inception***

In [ ]:
history_a1_in = inception_a1.fit(train_batch,
                                 epochs=50,
                                 validation_data=valid_batch,
                                 steps_per_epoch=256,
                                 callbacks=[reduce_lr])

pd.DataFrame(history_a1_in.history).to_pickle('history_a1_in.pkl')

In [ ]:
history_a2_in = inception_a2.fit(train_batch,
                                 epochs=50,
                                 validation_data=valid_batch,
                                 steps_per_epoch=256,
                                 callbacks=[reduce_lr])

pd.DataFrame(history_a2_in.history).to_pickle('history_a2_in.pkl')

In [ ]:
history_a3_in = inception_a3.fit(train_batch,
                                 epochs=50,
                                 validation_data=valid_batch,
                                 steps_per_epoch=256,
                                 callbacks=[reduce_lr])

pd.DataFrame(history_a3_in.history).to_pickle('history_a3_in.pkl')

In [ ]:
history_a4_in = inception_a4.fit(train_batch,
                                 epochs=50,
                                 validation_data=valid_batch,
                                 steps_per_epoch=256,
                                 callbacks=[reduce_lr])

pd.DataFrame(history_a4_in.history).to_pickle('history_a4_in.pkl')

In [ ]:
history_a5_in = inception_a5.fit(train_batch,
                                 epochs=50,
                                 validation_data=valid_batch,
                                 steps_per_epoch=256,
                                 callbacks=[reduce_lr])

pd.DataFrame(history_a5_in.history).to_pickle('history_a5_in.pkl')